In [ ]:
!unzip cat_dataset.zip
!unzip image_test.zip

In [ ]:
!nvidia-smi

Mon May  3 08:25:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    71W / 149W |    126MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from DaisyModule import *
import os
import tensorflow as tf
from glob import glob
import PIL
import numpy as np


In [ ]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# Optimisation, décommenter les lignes suivantes si on utilise un GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
#Définir les paramètres
cat_dataset = "./Cat_Dataset"
save_directory = "./Caches"
#Nombre d'images chargées dans le dataset à la fois. Garder ce nombre entre 16 et 64 si possible
batch_size = 32
image_height = 160
image_width = 160
#Taux d'apprentissage. Il est conseillé de le laisser à cette valeur pour éviter l'overfitting. On le changera plus tard
base_learning_rate = 0.0001
# Le nombre de fois the l'algorithme d'apprentissage va travailler à travers le dataset au complet. Mettre une valeur raisonnable, pour éviter l'overfitting
epochs = 40
# Le nombre de couches inférieures à garder bloquer. Le laisser à 100 avec MobilenetV2, et environ à 5 avec son modèle à soi, dépendamment de son nombre de couches.
layer = 100
num_classes = len(glob(cat_dataset + "/*"))
class_names = sorted([i.replace(cat_dataset, "") for i in glob(cat_dataset+"/*")])



In [ ]:
# Création des datasets. Définit les paramètres, puis créée les datasets individuellement
dataset_parameters = make_dataset(cat_dataset, batch_size, image_height, image_width)
train_dataset = dataset_parameters.train()
validation_dataset = dataset_parameters.val()

Found 2400 files belonging to 12 classes.
Using 1920 files for training.
Found 2400 files belonging to 12 classes.
Using 480 files for validation.


In [ ]:
#Création du modèle. Ici on utilise MobileNetV2, mais on peut le changer pour son propre dataset
model, base_model = make_model(image_height, image_width, train_dataset, num_classes).load_v2(base_learning_rate)
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
________________________________________________________________

In [ ]:
#Entraînement du modèle. Si la précision d'entraînement est supérieure à la précision de validation, ça veut dire que le modèle a commencé à mémoriser 
#les images du dataset par coeur, et n'est donc pas capable de généraliser. On appelle ça du overfitting.
#Les solutions possibles sont, entre autres, augmenter le nombre d'images dans le dataset (et non pas la batch_size), et utiliser MobileNetV2, si ce n'est pas déjà fait.
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)
model.evaluate(validation_dataset)

Epoch 1/40
60/60 [==============================] - 42s 100ms/step - loss: 2.7342 - accuracy: 0.1301 - val_loss: 2.1910 - val_accuracy: 0.2479
Epoch 2/40
60/60 [==============================] - 10s 83ms/step - loss: 2.1617 - accuracy: 0.2719 - val_loss: 1.8236 - val_accuracy: 0.4208
Epoch 3/40
60/60 [==============================] - 10s 82ms/step - loss: 1.8344 - accuracy: 0.4173 - val_loss: 1.5616 - val_accuracy: 0.5042
Epoch 4/40
60/60 [==============================] - 10s 82ms/step - loss: 1.5567 - accuracy: 0.5082 - val_loss: 1.3654 - val_accuracy: 0.5938
Epoch 5/40
60/60 [==============================] - 10s 81ms/step - loss: 1.3971 - accuracy: 0.5540 - val_loss: 1.2261 - val_accuracy: 0.6333
Epoch 6/40
60/60 [==============================] - 10s 81ms/step - loss: 1.1859 - accuracy: 0.6398 - val_loss: 1.1129 - val_accuracy: 0.6604
Epoch 7/40
60/60 [==============================] - 10s 83ms/step - loss: 1.1234 - accuracy: 0.6663 - val_loss: 1.0315 - val_accuracy: 0.6833
Epoch

[0.49775028228759766, 0.8333333134651184]

In [ ]:
#Fine Tuning. Le modèle est recompilé avec un taux d'apprentissage plus bas, et ses couches supérieures deviennent modifiables. Cela nous fait gagner
#quelque pourcentages de précision de plus
model = compile_for_fine_tuning(model, base_model, base_learning_rate).unlock(layer)
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)
model.evaluate(validation_dataset)

Epoch 1/40
60/60 [==============================] - 19s 124ms/step - loss: 0.7941 - accuracy: 0.7496 - val_loss: 0.4955 - val_accuracy: 0.8229
Epoch 2/40
60/60 [==============================] - 12s 107ms/step - loss: 0.6768 - accuracy: 0.7773 - val_loss: 0.4901 - val_accuracy: 0.8292
Epoch 3/40
60/60 [==============================] - 11s 107ms/step - loss: 0.6295 - accuracy: 0.8008 - val_loss: 0.4866 - val_accuracy: 0.8313
Epoch 4/40
60/60 [==============================] - 11s 107ms/step - loss: 0.5305 - accuracy: 0.8296 - val_loss: 0.4758 - val_accuracy: 0.8333
Epoch 5/40
60/60 [==============================] - 11s 106ms/step - loss: 0.5035 - accuracy: 0.8274 - val_loss: 0.4660 - val_accuracy: 0.8417
Epoch 6/40
60/60 [==============================] - 11s 105ms/step - loss: 0.4551 - accuracy: 0.8460 - val_loss: 0.4645 - val_accuracy: 0.8417
Epoch 7/40
60/60 [==============================] - 11s 108ms/step - loss: 0.4349 - accuracy: 0.8591 - val_loss: 0.4581 - val_accuracy: 0.8417

[0.34788912534713745, 0.8583333492279053]

In [ ]:
#Sauvegarde du modèle dans un fichier. Il peut être importé plus tard, pour nous éviter de reentraîner un modèle à chaque fois.
model.save(save_directory)

INFO:tensorflow:Assets written to: ./Caches/assets


**Prediction Time**

In [ ]:
model = tf.keras.models.load_model(save_directory)

for path in sorted(glob("./Image_Test/*")):
    img = tf.keras.preprocessing.image.load_img(path, target_size=(image_height, image_width, 3))

    #Transforme l'image en array, où chaque pixel est une liste, avec 3 valeurs RBG
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    #Prédit la ressemblance de l'image avec chaqu'une des classes qu'elle connaît
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])


    print(f"L'Image {path} appartient sûrement aux {class_names[np.argmax(score)]} avec {round(np.max(score)*100, 2)}% de sûreté")

L'Image ./Image_Test/Colin_test2.png appartient sûrement aux /Persian avec 99.68% de sûreté
L'Image ./Image_Test/Colin_test3.png appartient sûrement aux /Persian avec 98.06% de sûreté
L'Image ./Image_Test/birman1.jpg appartient sûrement aux /Birman avec 96.74% de sûreté
L'Image ./Image_Test/birman2.jpg appartient sûrement aux /Ragdoll avec 87.27% de sûreté
L'Image ./Image_Test/birman3.jpg appartient sûrement aux /Ragdoll avec 83.92% de sûreté
L'Image ./Image_Test/birman4.jpg appartient sûrement aux /Ragdoll avec 58.67% de sûreté
L'Image ./Image_Test/birman5.jpg appartient sûrement aux /Birman avec 85.97% de sûreté
L'Image ./Image_Test/colin.png appartient sûrement aux /Persian avec 48.71% de sûreté
L'Image ./Image_Test/demon.png appartient sûrement aux /Sphynx avec 99.39% de sûreté
L'Image ./Image_Test/edgar.jpg appartient sûrement aux /Russian_Blue avec 99.64% de sûreté
L'Image ./Image_Test/kael.png appartient sûrement aux /Persian avec 99.91% de sûreté
L'Image ./Image_Test/kael2.png 